# **Pré-requisitos**

In [1]:
# Instalar dependências do python
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importar bibliotecas necessárias para este projeto
from bs4 import BeautifulSoup
import pandas as pd
import re
import string

In [3]:
# Permitir visualizar toda a ementa das leis
pd.set_option('display.max_colwidth', None)

# **Carregando os dados**
Para este trabalho, vamos utilizar somente as ementas das leis.

In [4]:
# Carregar a base de dados mais atual
df = pd.read_csv('data/Leis de Sorocaba.csv')[['summary_tag']]
df.head()

,summary_tag
0,"<h1>(Dispõe sobre denominação de ""JOSÉ ANTONIO PASCOTO"" a uma via pública).</h1>"
1,"<h1>(Dispõe sobre denominação de ""Vitor Hage"" a uma via pública e dá outras providências).</h1>"
2,"<h1>(Dispõe sobre denominação de ""LISARDO CUNHA DIAS"" a uma via pública e dá outras providências).</h1>"
3,"<h1>(Dispõe sobre denominação de ""Desirre Ferraz Cardoso"" a uma via pública de nossa cidade e dá outras providências).</h1>"
4,"<h1>(Institui, no calendário oficial do Município o Dia do Sociólogo e dá outras providências).</h1>"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12812 entries, 0 to 12811
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   summary_tag  12777 non-null  object
dtypes: object(1)
memory usage: 100.2+ KB


Temos observações faltantes na coluna `summary_tag`.

É importante tratar isso quando formos limpar o texto.

# **Limpando o HTML**

Como vimos que existem informações de trechos revogados das leis em tags `<s>`. Vamos precisar removê-los antes.

In [6]:
# Regex para buscar títulos onde existe a tag <s>
regex = r'<s>'

# Filtrar onde a coluna 'summary_tag' contém o regex
mask = df['summary_tag'].str.contains(regex, regex=True, na=False)

In [7]:
df[mask].head(1)

,summary_tag
439,"<h1><s>(Dispõe sobre o credenciamento de administradoras de planos de saúde aos servidores contratados sob o regime da Consolidação das Leis do Trabalho - CLT e seus dependentes e dá outras providências).</s> <b>Dispõe sobre o credenciamento de administradoras e operadoras de planos de saúde aos servidores contratados sob o regime da Consolidação das Leis do Trabalho - CLT e Conselheiros Tutelares e seus dependentes e dá outras providência. (Redação dada pela Lei nº <a class=""link_law"" data-id=""7711952"" data-original-title="" Data da Norma: 08.04.2022 - (Altera a ementa e a redação do artigo 1º, da Lei nº 12.373, de 20 de setembro de 2021 que dispõe sobre o credenciamento de administradoras de planos de saúde aos servidores contratados sob o regime da Consolidação das Leis do Trabalho - CLT e seus dependentes e dá outras providências)."" data-toggle=""tooltip"" href=""https://leismunicipais.com.br/a1/sp/s/sorocaba/lei-ordinaria/2022/1253/12536/lei-ordinaria-n-12536-2022-altera-a-ementa-e-a-redacao-do-artigo-1-da-lei-n-12373-de-20-de-setembro-de-2021-que-dispoe-sobre-o-credenciamento-de-administradoras-de-planos-de-saude-aos-servidores-contratados-sob-o-regime-da-consolidacao-das-leis-do-trabalho-clt-e-seus-dependentes-e-da-outras-providencias"" rel=""tooltip"">12.536</a>/2022)</b></h1>"


In [8]:
def clean_html(html):
    # Verifica se o valor é NaN (observação faltante)
    # Retorna uma string vazia, caso seja NaN
    if pd.isnull(html):
        return ''

    soup = BeautifulSoup(html, 'html.parser')

    # Remove as tags <s> e seu conteúdo
    s_tags = soup.find_all('s')
    for s_tag in s_tags:
        s_tag.decompose()

    # Transforma o html em texto
    # Remove espaços no início e fim de cada tag
    # Preserva espaços entre as tags
    text = soup.get_text(separator=' ', strip=True) 

    return text.strip()

Vamos testar na observação 439, que anteriormente vimos ter tags `<s>`

In [9]:
print(df.loc[439, 'summary_tag'])
print(clean_html(df.loc[439, 'summary_tag']))

<h1><s>(Dispõe sobre o credenciamento de administradoras de planos de saúde aos servidores contratados sob o regime da Consolidação das Leis do Trabalho - CLT e seus dependentes e dá outras providências).</s> <b>Dispõe sobre o credenciamento de administradoras e operadoras de planos de saúde aos servidores contratados sob o regime da Consolidação das Leis do Trabalho - CLT e Conselheiros Tutelares e seus dependentes e dá outras providência. (Redação dada pela Lei nº <a class="link_law" data-id="7711952" data-original-title=" Data da Norma: 08.04.2022 - (Altera a ementa e a redação do artigo 1º, da Lei nº 12.373, de 20 de setembro de 2021 que dispõe sobre o credenciamento de administradoras de planos de saúde aos servidores contratados sob o regime da Consolidação das Leis do Trabalho - CLT e seus dependentes e dá outras providências)." data-toggle="tooltip" href="https://leismunicipais.com.br/a1/sp/s/sorocaba/lei-ordinaria/2022/1253/12536/lei-ordinaria-n-12536-2022-altera-a-ementa-e-a-

Vamos agora aplicar em todo o dataset.

In [10]:
df_clean = df.applymap(clean_html)
df_clean.columns = ['summary']
df_clean.head()

,summary
0,"(Dispõe sobre denominação de ""JOSÉ ANTONIO PASCOTO"" a uma via pública)."
1,"(Dispõe sobre denominação de ""Vitor Hage"" a uma via pública e dá outras providências)."
2,"(Dispõe sobre denominação de ""LISARDO CUNHA DIAS"" a uma via pública e dá outras providências)."
3,"(Dispõe sobre denominação de ""Desirre Ferraz Cardoso"" a uma via pública de nossa cidade e dá outras providências)."
4,"(Institui, no calendário oficial do Município o Dia do Sociólogo e dá outras providências)."


# **Remoção de caracteres especiais desnecessários**

Neste caso, vamos ter que avaliar caso a caso, pois existem pontuações que adicionam contexto e outras que podem ser removidas.

Além disso, é adequado padronizar diferentes notações encontradas em textos diferentes, de forma a facilitar o processamento de máquina.

In [11]:
# Verificando alguns caracteres especiais
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
# Verificando a quantidade de caracteres especiais
special_chars = string.punctuation + '§ºª°´'
for c in special_chars:
    print(c, sum(df_clean['summary'].str.contains(c, regex=False)))

! 0
" 5920
# 0
$ 61
% 13
& 5
' 0
( 1083
) 1135
* 3
+ 0
, 5536
- 1178
. 12496
/ 283
: 14
; 52
< 0
= 0
> 0
? 0
@ 0
[ 0
\ 0
] 0
^ 0
_ 2
` 48
{ 0
| 0
} 0
~ 0
§ 132
º 2124
ª 27
° 0
´ 16


Os caracteres que não agregam nenhum sentido e podem ser removidos sem tratamento são:

- `"`: Aspas duplas
- `(`: Parêntese aberto
- `)`: Parêntese fechado
- `*`: Asterisco
- `:`: Dois pontos
- `;`: Ponto e vírgula
- `_`: Sublinhado
- `: Acento grave
- `ª`: Símbolo ordinal feminino
- `´`: Acento agudo

Assim, os caracteres especiais que precisamos tratar são:

- `$`: Símbolo de cifrão
- `&`: E comercial
- `%`: Símbolo de porcentagem
- `-`: Hífen ou traço
- `,`: Vírgula
- `.`: Ponto
- `/`: Barra
- `§`: Símbolo de seção ou parágrafo
- `º`: Símbolo ordinal masculino

In [13]:
def remove_unnecessary_chars(text):
    return re.sub(r'["()*:;_`ª´]', ' ', text)

## **Símbolo de Cifrão (`$`)**

O primeiro caractere especial que deve ser tratado com atenção é o cifrão (`$`), que indica valores monetários. Como ele adiciona contexto nos textos legislativos, vamos mantê-los.

Porém, seria interessante padronizar os valores, de forma a facilitar o processamento.

In [14]:
mask = df_clean['summary'].str.contains(r'\$', regex=True)
df_clean.loc[mask].sample(5, random_state=42)

,summary
1566,"ESTIMA A RECEITA E FIXA A DESPESA DO MUNICÍPIO PARA O EXERCÍCIO DE 2016. (R$ 2.707.495.391,00)"
3398,"ESTIMA A RECEITA E FIXA A DESPESA DO MUNICÍPIO PARA O EXERCÍCIO DE 2011.(R$ 1.500.247.500,00)"
12183,"DESTINA VERBA DE CR$ 200.000,00 (DUZENTOS MIL CRUZEIROS) À ASSOCIAÇÃO SOROCABANA DE AÇÃO SOCIAL - ASAS"
11845,"DISPÕE SÔBRE ABERTURA DE CRÉDITO ESPECIAL DE CR$ 309.966,80."
10653,"DISPÕE SOBRE AUTORIZAÇÃO PARA O EXECUTIVO CONTRAIR EMPRÉSTIMOS, NO VALOR DE ATÉ CR$ 300.000.000,00."


**O que podemos observar:**
- Existem moedas diferentes
- Os valores numéricos possuem separadores de casas de milhares e centavos *
- Há um espaço entre o cifrão e o valor numérico *
- Alguns textos informam o valor escrito por extenso, após os numerais

\* É preciso verificar se as regras são consistentes em todos os exemplos

In [15]:
print('Moedas existentes')
print('Na ementa:', df_clean['summary'].str.extract(r'(\w*\$)').dropna()[0].unique())

Moedas existentes
Na ementa: ['R$' 'NCZ$' 'CZ$' 'CR$' 'NCR$']


In [16]:
print('Total:', sum(df_clean['summary'].str.contains(r'\$', regex=True)))
print()
print('Sem espaço:', sum(df_clean['summary'].str.contains(r'\$\d', regex=True)))
print('Com espaço:', sum(df_clean['summary'].str.contains(r'\$\s\d', regex=True)))
print()
print('Com separador de milhar:', sum(df_clean['summary'].str.contains(r'\$\s?\d+\.\d+', regex=True)))
print('Com separador de centavos:', sum(df_clean['summary'].str.contains(r'\$\s?\d+(?:\.\d+)*\,\d+', regex=True)))

Total: 61

Sem espaço: 13
Com espaço: 48

Com separador de milhar: 58
Com separador de centavos: 59


In [17]:
# Sem separador de milhar
contains_dollar = df_clean['summary'].str.contains(r'\$', regex=True)
contains_thousand_separator = df_clean['summary'].str.contains(r'\$\s?\d+\.\d+', regex=True)
contains_cent_separator = df_clean['summary'].str.contains(r'\$\s?\d+(?:\.\d+)*,\d+', regex=True)

print('Sem separador de milhar:', sum(contains_dollar & ~contains_thousand_separator))
display(df_clean[contains_dollar & ~contains_thousand_separator][['summary']])

print('Sem separador de centavos:', sum(contains_dollar & ~contains_cent_separator))
display(df_clean[contains_dollar & ~contains_cent_separator][['summary']])

Sem separador de milhar: 3


,summary
12059,"ELEVA PARA CR$ 600,00 (SEISCENTOS CRUZEIROS) O SALÁRIO FAMÍLIA."
12226,"ELEVA PARA CR$ 300,00 O SALÁRIO -FAMÍLIA."
12714,"AUTORIZA O CANCELAMENTO DE DÉBITOS FISCAIS ATÉ A IMPORTÂNCIA DE CR$100,00."


Sem separador de centavos: 2


,summary
11475,DISPÕE SÔBRE UM EMPRÉSTIMO DE CR$159.836.356 A SER CONTRAÍDO COM A CAIXA ECONÔMICA DO ESTADO DE SÃO PAULO.
11477,DISPÕE SÔBRE ABERTURA DE CRÉDITO ESPECIAL NO VALOR DE CR$8.106.322.


In [18]:
contains_written_monetary_value = df_clean['summary'].str.contains(r'\w*\$\s?\d+(?:\.\d+)*(?:,\d+)?[(\s\w,]* (?:cruzados?|cruzeiros?|rea(?:l|is))', regex=True, case=False)
print('Com valor escrito:', sum(contains_written_monetary_value))
display(df_clean[contains_written_monetary_value][['summary']])

Com valor escrito: 16


,summary
4175,"ESTIMA A RECEITA E FIXA A DESPESA DO MUNICÍPIO PARA O EXERCÍCIO DE 2009. (R$ 1.150.387.700,00 UM BILHÃO, CENTO E CINQUENTA MILHÕES, TREZENTOS E OITENTA E SETE MIL E SETECENTOS REAIS)"
5238,"DISPÕE SOBRE AUTORIZAÇÃO PARA ABERTURA DE CRÉDITO ADICIONAL ESPECIAL, ATÉ O LIMITE DE R$ 1.500.000,00 (HUM MILHÃO E QUINHENTOS MIL REAIS), PARA CONCLUSÃO E ADAPTAÇÃO DO CENTRO DE ATENDIMENTO A MULHER E DE REFERÊNCIA CIRÚRGICA DA POLICLÍNICA MUNICIPAL E DÁ OUTRAS PROVIDÊNCIAS."
10241,"DISPÕE SOBRE CRIAÇÃO DE UM FUNDO HABITACIONAL, ABRE UM CRÉDITO ESPECIAL DE CZ$ 3.000.000,00 (TRÊS MILHÕES DE CRUZADOS) E DÁ OUTRAS PROVIDÊNCIAS."
10636,"DISPÕE SOBRE A CONCESSÃO DE AUXÍLIO DE CR$ 800.000,00 (OITOCENTOS MIL CRUZEIROS) AO CENTRO DE CIÊNCIAS MÉDICAS E BIOLÓGICAS DE SOROCABA."
10645,"DISPÕE SOBRE CONCESSÃO DE AUXÍLIO DE CR$ 700.000,00 (SETECENTOS MIL CRUZEIROS) À IRMANDADE DA SANTA CASA DE MISERICÓRDIA DE SOROCABA."
10683,"DISPÕE SOBRE EMPRÉSTIMO A SER CONTRAÍDO COM A CAIXA ECONÔMICA DO ESTADO DE SÃO PAULO S/A ATÉ O VALOR DE CR$ 15.000.000,00 (QUINZE MILHÕES DE CRUZEIROS) PARA REFORMA DO PALÁCIO BRIGADEIRO RAPHAEL TOBIAS DE AGUIAR."
11013,"AUTORIZA O PODER EXECUTIVO A CONTRATAR OPERAÇÃO DE CRÉDITO ATÉ A IMPORTÂNCIA DE CR$50.000.000,00 (CINQUENTA MILHÕES DE CRUZEIROS) E DÁ OUTRAS PROVIDÊNCIAS."
11895,"AUTORIZA A PREFEITURA MUNICIPAL DE SOROCABA A AVALIZAR UMA NOTA PROMISSÓRIA NO VALOR DE CR$ 60.000.000,00 (SESSENTA MILHÕES DE CRUZEIROS) EM FAVOR DA C.R.T. SOROCABANA, PARA O BANCO DO ESTADO DE SÃO PAULO, NAS CONDIÇÕES QUE MENCIONA."
12059,"ELEVA PARA CR$ 600,00 (SEISCENTOS CRUZEIROS) O SALÁRIO FAMÍLIA."
12154,"DISPÕE SÔBRE CONCESSÃO DE AUXÍLIO DE CR$ 50.000,00 (CINQUENTA MIL CRUZEIROS), AO CENTRO ACADÊMICO ""SANTO THOMAZ DE AQUINO"" E DÁ OUTRAS PROVIDÊNCIAS."


### **O que vamos fazer:**

- Manter as diferentes moedas
- Transformar todos os valores numéricos para o token "`TOKEN_VALOR`", dado que os números em si não importam para a nossa análise.
- Remover os valores escritos por extenso

In [19]:

def normalize_money(text):
    pattern = r"(?P<moeda>\w+\$)\s?(?P<valor>\d+(?:\.\d+)*(?:,\d+)?)(?:(?P<extenso>[(\s\w,]* (?:cruzados?|cruzeiros?|rea(?:l|is))(?: e [\s\w]* centavos?)?\)?))?"
    repl = lambda m: ' $ TOKEN_VALOR '
    return re.sub(pattern, repl, text, flags=re.IGNORECASE)

In [20]:
# Teste de conversão
df_clean.loc[contains_dollar, 'summary'].apply(normalize_money).sample(5, random_state=42).to_frame()

,summary
1566,ESTIMA A RECEITA E FIXA A DESPESA DO MUNICÍPIO PARA O EXERCÍCIO DE 2016. ( $ TOKEN_VALOR )
3398,ESTIMA A RECEITA E FIXA A DESPESA DO MUNICÍPIO PARA O EXERCÍCIO DE 2011.( $ TOKEN_VALOR )
12183,DESTINA VERBA DE $ TOKEN_VALOR À ASSOCIAÇÃO SOROCABANA DE AÇÃO SOCIAL - ASAS
11845,DISPÕE SÔBRE ABERTURA DE CRÉDITO ESPECIAL DE $ TOKEN_VALOR .
10653,"DISPÕE SOBRE AUTORIZAÇÃO PARA O EXECUTIVO CONTRAIR EMPRÉSTIMOS, NO VALOR DE ATÉ $ TOKEN_VALOR ."


## **Símbolo de Porcentagem (`%`)**

Os números percentuais adicionam algum contexto, principalmente relacionado a tributos, gratificações e estabelecimento de cotas, nos textos legislativos. Assim, vamos mantê-los.

In [21]:
mask = df_clean['summary'].str.contains(r'%', regex=True)
df_clean.loc[mask]

,summary
182,Dispõe sobre desconto de 5% sobre o Imposto Predial e Territorial Urbano - IPTU aos proprietários de imóveis urbanos que sejam doadores de sangue e dá outras providências.
946,Estabelece desconto de 15% (quinze por cento) no pagamento de IPTU - Imposto sobre Propriedade Territorial Urbana dos imóveis lindeiros localizados nas margens da Rodovia Raposo Tavares no âmbito do município de Sorocaba e dá outras providências.
1108,Estabelece desconto de 15% (quinze por cento) no pagamento de IPTU - Imposto sobre Propriedade Territorial Urbana dos imóveis localizados nas vias públicas onde se realizam as feiras-livres no âmbito do Município de Sorocaba e dá outras providências.
1713,"ESTABELECE RESERVA DE, NO MÍNIMO, 1% (UM POR CENTO) DESTINADO AOS SERVIDORES PÚBLICOS NO ÂMBITO DO MUNICÍPIO DE SOROCABA, PARA A AQUISIÇÃO DE IMÓVEIS PELO ""PROGRAMA MINHA CASA MINHA VIDA"" E DEMAIS PROJETOS DE HABITAÇÃO PROMOVIDOS PELO MUNICÍPIO DE SOROCABA, E DÁ OUTRAS PROVIDÊNCIAS."
1936,"TORNA OBRIGATÓRIA A RESERVA DE 5% (CINCO POR CENTO) DE MESAS E CADEIRAS PARA IDOSOS, PESSOAS COM DEFICIÊNCIA E PARA MULHERES GESTANTES NAS PRAÇAS DE ALIMENTAÇÃO NOS SHOPPINGS CENTERS E GALERIAS, NO MUNICÍPIO DE SOROCABA, E DÁ OUTRAS PROVIDÊNCIAS."
6913,"DISPÕE SOBRE RESERVAR 5%, PARA SORTEIO DE LOTES, CASAS E APARTAMENTOS POPULARES, NOS CASOS EM QUE ESPECIFICA E DÁ OUTRAS PROVIDÊNCIAS."
8245,"CONCEDE ÀS PESSOAS COM 60 ANOS OU MAIS, O DIREITO DE ADQUIRIR COM 50% (CINQUENTA POR CENTO) DE DESCONTO NO PREÇO DE INGRESSOS PARA CINEMAS, CINECLUBES, TEATROS, EVENTOS ESPORTIVOS, ESPETÁCULOS CIRCENSES E MUSICAIS E DÁ OUTRAS PROVIDÊNCIAS."
11602,"DISPÕE SÔBRE OBRIGATORIEDADE DA APLICAÇÃO PELO MUNICÍPIO, DE 50% DA RENDA DE CADA DISTRITO, NO PRÓPRIO DISTRITO, E DÁ OUTRAS PROVIDÊNCIAS."
11614,"DISPÕE SÔBRE CONCESSÃO DE ADICIONAL ESPECIAL DE 30% AOS SERVIDORES E FUNCIONÁRIOS PÚBLICOS MUNICIPAIS QUE PRESTAM SERVIÇOS DE NATUREZA INSALUBRE, E DÁ OUTRAS PROVIDÊNCIAS."
12010,"DISPÕE SÔBRE GRATIFICAÇÃO DE 30% AOS SERVIDORES DA LIMPEZA PÚBLICA, E DÁ OUTRAS PROVIDÊNCIAS."


## **E Comercial (`&`)**

É normalmente utilizado como conector de nomes, em marcas. Pode ser removido sem prejuízo ao contexto, se estiver isolado.

In [22]:
# Isolado
mask = df_clean['summary'].str.contains(r'\s&\s', regex=True)
df_clean.loc[mask]

,summary
2183,"DECLARA DE UTILIDADE PÚBLICA A ""ASSOCIAÇÃO SOROCABA E REGIÃO CONVENTION & VISITORS BUREAU"")."
5099,"AUTORIZA A PREFEITURA MUNICIPAL DE SOROCABA A CELEBRAR CONVÊNIO COM O IEE - INSTITUTO ESPORTE & EDUCAÇÃO, ATRAVÉS DAS SECRETARIAS DE ESPORTES E LAZER; DA EDUCAÇÃO E DE PARCERIAS, VISANDO A CAPACITAÇÃO DE PROFESSORES DE EDUCAÇÃO FÍSICA DA REDE PÚBLICA MUNICIPAL DE ENSINO E DOS PRÓPRIOS ESPORTIVOS E DÁ OUTRAS PROVIDÊNCIAS."
11091,"AUTORIZA A DOAÇÃO DE IMÓVEL À FIRMA ""DOMENICO BESTETTI & CIA LTDA."", E DÁ OUTRAS PROVIDÊNCIAS."
11120,"AUTORIZA A DOAÇÃO DE IMÓVEL PARA A FIRMA ""NICHOLSON K & F DO BRASIL S.A INDÚSTRIA E COMÉRCIO"" E DÁ OUTRAS PROVIDÊNCIAS."


In [23]:
# Não isolado
mask = df_clean['summary'].str.contains(r'\S&\S', regex=True)
df_clean.loc[mask]

,summary
4174,"AUTORIZA O MUNICÍPIO DE SOROCABA A CELEBRAR CONVÊNIO COM O GOVERNO DO ESTADO DE SÃO PAULO, POR INTERMÉDIO DA SECRETARIA DE DESENVOLVIMENTO, OBJETIVANDO A TRANSFERÊNCIA DE RECURSOS FINANCEIROS PARA A REALIZAÇÃO DO PROJETO ""DESENVOLVIMENTO DE ESTUDOS PARA ESTABELECER AS BASES DO PLANO DE C&T&I DO PARQUE TECNOLÓGICO DE SOROCABA"", E DÁ OUTRAS PROVIDÊNCIAS)."


In [24]:
def normalize_ampersand(text):
    return text.replace(' & ', ' ')

In [25]:
# Testando a remoção de &
mask = df_clean['summary'].str.contains(r'&', regex=True)
df_clean.loc[mask, 'summary'].apply(normalize_ampersand).to_frame()

,summary
2183,"DECLARA DE UTILIDADE PÚBLICA A ""ASSOCIAÇÃO SOROCABA E REGIÃO CONVENTION VISITORS BUREAU"")."
4174,"AUTORIZA O MUNICÍPIO DE SOROCABA A CELEBRAR CONVÊNIO COM O GOVERNO DO ESTADO DE SÃO PAULO, POR INTERMÉDIO DA SECRETARIA DE DESENVOLVIMENTO, OBJETIVANDO A TRANSFERÊNCIA DE RECURSOS FINANCEIROS PARA A REALIZAÇÃO DO PROJETO ""DESENVOLVIMENTO DE ESTUDOS PARA ESTABELECER AS BASES DO PLANO DE C&T&I DO PARQUE TECNOLÓGICO DE SOROCABA"", E DÁ OUTRAS PROVIDÊNCIAS)."
5099,"AUTORIZA A PREFEITURA MUNICIPAL DE SOROCABA A CELEBRAR CONVÊNIO COM O IEE - INSTITUTO ESPORTE EDUCAÇÃO, ATRAVÉS DAS SECRETARIAS DE ESPORTES E LAZER; DA EDUCAÇÃO E DE PARCERIAS, VISANDO A CAPACITAÇÃO DE PROFESSORES DE EDUCAÇÃO FÍSICA DA REDE PÚBLICA MUNICIPAL DE ENSINO E DOS PRÓPRIOS ESPORTIVOS E DÁ OUTRAS PROVIDÊNCIAS."
11091,"AUTORIZA A DOAÇÃO DE IMÓVEL À FIRMA ""DOMENICO BESTETTI CIA LTDA."", E DÁ OUTRAS PROVIDÊNCIAS."
11120,"AUTORIZA A DOAÇÃO DE IMÓVEL PARA A FIRMA ""NICHOLSON K F DO BRASIL S.A INDÚSTRIA E COMÉRCIO"" E DÁ OUTRAS PROVIDÊNCIAS."


## **Vírgula (`,`)**

É usado como separador de palavras ou casas decimais. Não agrega contexto no primeiro caso, mas no segundo, sim. 

Como já tratamos os valores monetários, vamos verificar somente os casos onde não há cifrão.

In [26]:
mask = df_clean['summary'].str.contains(r'(?:\d,\D)|(?:\D,\d)|(?:\D,\D)', regex=True)
df_clean.loc[mask]

,summary
4,"(Institui, no calendário oficial do Município o Dia do Sociólogo e dá outras providências)."
5,"(Altera a redação dos artigos 7º, 8º, 9º e 10, da Lei Municipal nº 11.982 , de 14 de maio de 2019 e dá outras providências)."
6,"(Institui como Patrimônio Cultural Imaterial da cidade de Sorocaba a Orquestra Sinfônica de Sorocaba (OSS), e dá outras providências)."
7,"(Institui e insere no calendário oficial do Município de Sorocaba a semana municipal do dia 22 de abril como a Semana Municipal de Conscientização da Doença de Parkinson, denominada ""Tulipa Vermelha"")."
9,"(Institui a ""Plataforma CUR - Canal Unificado de Remédios"" no âmbito do Município de Sorocaba, e dá outras providências)."
...,...
12777,"DISPÕE SÔBRE O ESTABELECIMENTO DE UMA ""FAIXA COMERCIAL CENTRAL"", PARA EFEITO DA MELHORIA ARQUITETÔNICA DA CIDADE."
12781,"AUTORIZA A VENDA, MEDIANTE CONCORRÊNCIA PÚBLICA, DE MATERIAIS DE PROPRIEDADE DE MUNICIPALIDADE."
12786,"DISPÕE SÔBRE CONSTRUÇÃO E REFORMA DE MUROS, GRADIS E PASSEIOS, E DÁ OUTRAS PROVIDÊNCIAS."
12792,"DÁ NOVA REDAÇÃO AO ITEM II, DO ARTIGO 2º, DO DECRETO-LEI N. 168, DE 31/12/1.946."


Existem casos onde a vírgula foi acrescentada entre palavras, faltando um espaço. Isso é um erro de digitação.

Quando formos remover as vírgulas, precisamos acrescentar um espaço.

In [27]:
mask = df_clean['summary'].str.contains(r'[A-Za-z],[A-Za-z]', regex=True)
df_clean.loc[mask]

,summary
7271,"OBRIGA AS SALAS DE ESPETÁCULOS, CULTURAIS, OS TEATROS, CINEMAS, ANFITEATROS, BIBLIOTECAS, GINÁSIOS ESPORTIVOS E OUTROS LOCAIS,INCLUSIVE OS ESTABELECIMENTOS DE ENSINO, QUE DISPONHAM DE POLTRONAS FIXAS, A MANTER EM SUAS DEPENDÊNCIAS ESPAÇOS PRIVATIVOS, DESTINADOS A USUÁRIOS DE CADEIRAS DE RODA E DÁ OUTRAS PROVIDÊNCIAS."
7406,"AUTORIZA A PREFEITURA MUNICIPAL DE SOROCABA A CELEBRAR CONVÊNIO COM A SECRETARIA DE AGRICULTURA E ABASTECIMENTO DO ESTADO DE SÃO PAULO,PARA EXECUÇÃO DO PROGRAMA CAMPO/CIDADE LEITE E DÁ OUTRAS PROVIDÊNCIAS."
7518,"ACRESCENTA E ALTERA ARTIGOS,PARÁGRAFOS E INCISOS JUNTO A LEI Nº 3.800 , DE 02 DE DEZEMBRO DE 1 991, BEM COMO DÁ OUTRAS PROVIDÊNCIAS."
8173,"DISPÕE SOBRE DENOMINAÇÃO DE CONJUNTO HABITACIONAL ""DR.ULISSES GUIMARÃES"",A UNIDADE SOROCABA B-I,CONSTRUÍDA PELA COMPANHIA DE DESENVOLVIMENTO HABITACIONAL E URBANO DO ESTADO DE SÃO PAULO NO PARQUE ""VITÓRIA RÉGIA"" E DÁ OUTRAS PROVIDÊNCIAS."
8948,"AUTORIZA A ALIENAÇÃO A PROPRIETÁRIOS LINDEIROS,ATRAVÉS DE SOLICITAÇÃO DE PARTE O LOTE 01, QUADRA ""J"" DA VILA SANTA TEREZA E DÁ OUTRAS PROVIDÊNCIAS."
8968,"DISPÕE SOBRE DESAFETAÇÃO DE BEM PÚBLICO DE USO COMUM E ESPECIAL,PASSANDO A INTEGRAR O ROL, DOS BENS DOMINAIS DO MUNICÍPIO E CONCEDE DIREITO REAL DE USO AO SINDICATO PROFISSIONAL DOS FEIRANTES E VENDEDORES AMBULANTES DE SOROCABA E REGIÃO E DÁ OUTRAS, PROVIDÊNCIAS."
9091,"DISPÕE SOBRE AUTORIZAÇÃO PARA A PREFEITURA MUNICIPAL RECEBER SOB A FORMA DE DOAÇÃO COM ENCARGOS,IMÓVEL DESTINADO À IMPLANTAÇÃO DE VIA PÚBLICA A DÁ OUTRAS PROVIDÊNCIAS."
9169,"DISPÕE SOBRE DENOMINAÇÃO DE ROBERT STEPHENSON SMITH BADEN POWELL,A UM PRÓPRIO PÚBLICO DE NOSSA CIDADE."
9225,"DECLARA DE UTILIDADE,PÚBLICA O ""ESQUADRÃO PARA RECUPERAR VIDAS - MOVIMENTO PARA RECUPERAÇÃO HUMANA""."
9555,"AUTORIZA A PREFEITURA MUNICIPAL DE SOROCABA,CELEBRAR CONVÊNIO COM O CENTRO DE INTEGRAÇÃO EMPRESA-ESCOLA - CIEE, PARA A IMPLANTAÇÃO DE PROGRAMA RELACIONADO AO ESTUDANTE-ESTAGIÁRIO E DÁ OUTRAS PROVIDÊNCIAS."


Pelo o que podemos ver abaixo, também existem casos onde a vírgula foi inserida incorretamente no lugar do separador de milhar em datas.

Também temos um caso onde a vírgula é usada como separador decimal.

Podemos manter essas vírgulas e corrigir as datas posteriormente.

In [28]:
mask = df_clean['summary'].str.contains(r'[^\$] (?:\d+\.\d+)?\d+,\d+', regex=True)
df_clean.loc[mask]

,summary
8311,"ALTERA REDAÇÃO DO ARTIGO 6º DA LEI Nº 3.627 , DE 28 DE JUNHO DE 1,991, E DÁ OUTRAS PROVIDÊNCIAS."
9577,"SUPRIME O ALARGAMENTO PARA 25,00 METROS DA RUA APARECIDA, NOS TERMOS DO PARÁGRAFO ÚNICO, DO ARTIGO 3º, DA LEI Nº 1.438 , DE 21 DE NOVEMBRO DE 1.966."
10699,"DÁ NOVA REDAÇÃO AO ARTIGO 2º DA LEI Nº 2.021 , DE 9 DE AGOSTO DE 1,979."


In [29]:
def normalize_commas(text):
    # Encontra vírgulas em "\D,\d", "\d,\D", e "\D,\D"
    pattern = r'(\D),(\d)|(\d),(\D)|(\D),(\D)'
    
    # Substitui a vírgula por um espaço
    repl = lambda m: m.group().replace(',', ' ')
    
    return re.sub(pattern, repl, text)

In [30]:
# Teste de normalização de vírgulas
df_clean.loc[mask, 'summary'].apply(normalize_commas).to_frame()

,summary
8311,"ALTERA REDAÇÃO DO ARTIGO 6º DA LEI Nº 3.627 DE 28 DE JUNHO DE 1,991 E DÁ OUTRAS PROVIDÊNCIAS."
9577,"SUPRIME O ALARGAMENTO PARA 25,00 METROS DA RUA APARECIDA NOS TERMOS DO PARÁGRAFO ÚNICO DO ARTIGO 3º DA LEI Nº 1.438 DE 21 DE NOVEMBRO DE 1.966."
10699,"DÁ NOVA REDAÇÃO AO ARTIGO 2º DA LEI Nº 2.021 DE 9 DE AGOSTO DE 1,979."


## **Hífen ou traço (`-`)**

Quando usados como traços, ou seja, separador de palavras e sentenças, não agrega contexto e pode ser removido.

Quando usado como hífen, ou seja, conector de palavras, agrega contexto e não deve ser removido.

In [31]:
# Traço
mask = df_clean['summary'].str.contains(r'\s-\s\S-\s|\s-\S', regex=True)
df_clean.loc[mask]

,summary
4090,"ACRESCENTA O ART. 1º -A À LEI Nº 1.005 , DE 19 DE OUTUBRO DE 1962, QUE DISPÕE SOBRE CONCESSÃO DE AUXÍLIO ÀS MÃES, NAS CONDIÇÕES QUE MENCIONA E DÁ OUTRAS PROVIDÊNCIAS."
4967,"DISPÕE SOBRE DENOMINAÇÃO DE ""PARQUE KASATO -MARU"" A UMA ÁREA PÚBLICA DE NOSSA CIDADE E DÁ OUTRAS PROVIDÊNCIAS."
7410,DISPÕE SOBRE A CRIAÇÃO DO CONSELHO MUNICIPAL DE TRÂNSITO -COMUTRAN E DÁ OUTRAS PROVIDÊNCIAS.
7411,"AUTORIZA A PREFEITURA MUNICIPAL A CELEBRAR CONVÊNIO COM O GOVERNO DO ESTADO DE SÃO PAULO, ATRAVÉS DA SECRETARIA DA EDUCAÇÃO E A FUNDAÇÃO PARA A DESENVOLVIMENTO DA EDUCAÇÃO -FDE E DÁ OUTRAS PROVIDÊNCIAS."
7478,"DECLARA DE UTILIDADE PÚBLICA O ""CLUBE DOS RADIOAMADORES DE SOROCABA -CRASO""."
7585,"DISPÕE SOBRE DENOMINAÇÃO DE ""ARY MADUREIRA FILHO -(MARCOS CESAR)"", A UMA VIA PÚBLICA DE NOSSA CIDADE E DÁ OUTRAS PROVIDÊNCIAS"
7708,AUTORIZA A EMPRESA DE DESENVOLVIMENTO URBANO E SOCIAL DE SOROCABA -URBES A FINANCIAR DESPESAS DE ESCRITURA E REGISTRO DE IMÓVEIS E DÁ OUTRAS PROVIDÊNCIAS.
7767,"AUTORIZA A PREFEITURA MUNICIPAL A: FIRMAR CONVÊNIOS E TERMOS QUE OBJETIREM A EXECUÇÃO DE CONJUNTOS HABITACIONAIS NO MUNICÍPIO COM A COMPANHIA DE HABITAÇÃO POPULAR BANDEIRANTE -""COHAB-BANDEIRANTE, ORGÃO INTEGRANTE DO SISTEMA FINANCEIRO DE HABITAÇÃO, NA QUALIDADE AGENTE FINANCEIRO/PROMOTR/DE ASSESSORIA TÉCNICA/PARA ATIVIDADES COMPLEMENTARES; ASSUMIR OBRIGAÇÕES EM CONTRATOS DE EMPRESTIMO PARA CONSTRUÇÃO DE UNIDADES HABITACIONAIS POPULARES, PERANTE A CAIXA ECNONOMICA FEDERAL E/OU OUTROS ORGÃOS FINACIADORES AUTORIZADOS A OPERAR EM PROGRAMAS FEDERAIS DE HABITAÇÃO, E OBRIGAÇÕES PERANTE O FUNDO DE GARANTIA DO TEMPO DE SERVIÇO, NA QUALIDADE DE UMA DAS MUNICIPALIDADES ACIONISTAS/CONTROLADORAS DA ""COHAB-BANDEIRANTE"", APROVAR PROJETOS HABITACIONAIS ELABORADOS PELA ""COHAB-BANDEIRANTE"", COM REQUISITOS MÍNIMOS E CONDIÇÕES ESPECIAIS DE PARCELAMENTO E USO DO SOLO, PARA PROGRAMAS DE INTERESSE SOCIAL; APROVAR OS ATOS E PROCEDIMENTOS ADMINISTRATIVOS DA ""COHAB-BANDEIRANTE, EM CONFORMIDADE COM AS NORMAS GERAIS SOBRE LICITAÇÕES E CONTRATOS ADMINISTRATIVOS PERTINETES A OBRAS, SERVIÇOS, COMPRAS, ALIENAÇÕES E LOCAÇÕES DETERMINADAS PELO ARTIGO 37, INCISO XXI, DA CONSTITUIÇÃO FEDERAL, REGULAMENTADO PELA LEI Nº 8666, DE 21/06/93, E LEGISLAÇÃO POSTERIOR."
7887,"AMPLIA O VALOR DO REPASSE MENSAL DESTINADO A FUNDEC - FUNDAÇÃO DE DESENVOLVIMENTO CULTURAL DE SOROCABA -, E DÁ OUTRAS PROVIDÊNCIAS."
8535,"AUTORIZA A PREFEITURA MUNICIPAL A ADITAR O CONVÊNIO COM O GRUPO DE PESQUISA E ASSISTÊNCIA AO CÃNCER INFANTIL -G.P.A.C.I., E DÁ OUTRAS PROVIDÊNCIAS."


In [32]:
def normalize_dashes(text):
    return re.sub(r'\s-\s|\S-\s|\s-\S', ' ', text)

In [33]:
# Teste de remoção
mask = df_clean['summary'].str.contains(r'\s-\s', regex=True)
df_clean.loc[mask, 'summary'].apply(normalize_dashes).to_frame()

,summary
9,"(Institui a ""Plataforma CUR Canal Unificado de Remédios"" no âmbito do Município de Sorocaba, e dá outras providências)."
11,"(Declara de Utilidade Pública a ""CASA CATTANI Instituto Brasileiro de Assistência Apoio Humanização e Desenvolvimento Social"" e dá outras providências)."
17,(Institui o Programa de Regularização Fiscal do Município REFIS e dá outras providências).
23,"(Institui, no âmbito do Município de Sorocaba, o Programa de Melhorias Habitacionais para reforma de moradias inseridas em Zonas ou Áreas de Especial Interesse Social AEIS/ZEIS e dá outras providências)."
35,"(Inclui o art. 15-A na Lei nº 11.648 , de 22 de dezembro de 2017, que dispõe sobre a organização do Sistema Municipal de Defesa do Consumidor SMDC, institui o Conselho Municipal de Proteção e Defesa do Consumidor COMDECON e o Fundo Municipal de Proteção e Defesa do Consumidor FMDC, cria a superintendência do Serviço Municipal de Proteção ao Consumidor, revoga expressamente a Lei nº 2.072 , de 3 de junho de 1980 e dá outras providências)."
...,...
12229,"DECLARA DE UTILIDADE PÚBLICA A ""CAIXA DE SOCORROS DOS EMPREGADOS DA RODO FERROVIA E OPERÁRIOS DE SOROCABA""."
12464,DISPÕE SÔBRE CONCESSÃO DE SALÁRIO FAMILÍA ÀS VIÚVAS DOS SERVIDORES MUNICIPAIS.
12487,"DISPÕE SÔBRE ALTERAÇÃO DO ARTIGO 1º DA LEI N. 21 , DE 5/3/1948 E DO ARTIGO 308, DA LEI N.162, DE 18/8/1950 CÓDIGO DE OBRAS."
12498,DISPÕE SÔBRE AUMENTO DO SALÁRIO FAMÍLIA.


In [34]:
# Hífen
mask = df_clean['summary'].str.contains(r'\S-\S', regex=True)
df_clean.loc[mask]

,summary
22,"(Dispõe sobre a instituição de Área de Especial Interesse Social para Habitação (AEIS), na área pública remanescente denominada por I-D, da planta de desdobro elaborada pela Prefeitura Municipal de Sorocaba, situada na Avenida Itavuvu localizado no perímetro urbano desta cidade, conforme descrição constante na matrícula nº 125.524 registrada nº 1º Oficial de Registro de Imóveis de Sorocaba para promoção de Lotes Sociais e dá outras providências)."
35,"(Inclui o art. 15-A na Lei nº 11.648 , de 22 de dezembro de 2017, que dispõe sobre a organização do Sistema Municipal de Defesa do Consumidor - SMDC, institui o Conselho Municipal de Proteção e Defesa do Consumidor - COMDECON e o Fundo Municipal de Proteção e Defesa do Consumidor - FMDC, cria a superintendência do Serviço Municipal de Proteção ao Consumidor, revoga expressamente a Lei nº 2.072 , de 3 de junho de 1980 e dá outras providências)."
117,"(Institui o ""Mês do Jiu-Jitsu Brasileiro (BJJ)"" em Sorocaba e autoriza o ""Festival Sorocabano de Jiu-Jitsu Brasileiro"", e dá outras providências)."
133,"(Dispõe sobre a implantação de sistema de monitoramento de imagens nos estabelecimentos que comercializam ferros-velhos, sucatas e afins e dá outras providências)."
193,"(Institui a obrigatoriedade, por parte de condomínios residenciais ou comerciais, de comunicação às autoridades policiais e órgãos municipais competentes da ocorrência ou indícios de casos de maus-tratos ou quaisquer violações de direitos dos animais em suas unidades condominiais ou nas áreas comuns no Município de Sorocaba e dá outras providências)."
...,...
12453,"DISPÕE SÔBRE ABERTURA DE CONCORRÊNCIA PÚBLICA PARA EXPLORAÇÃO DOS SERVIÇOS DE TRANSPORTES COLETIVOS URBANOS E INTER-DISTRITAIS DO MUNICÍPIO, POR MEIO DE ÔNIBUS."
12740,DISPÕE SÔBRE CONCESSÃO DE SEPULTURAS PERPÉTUAS AOS EX-INTEGRANTES DAS FORÇAS EXPEDICIONÁRIAS BRASILEIRAS)
12746,"ALTERA O DISPOSTO NA LETRA ""A"" DO PARÁGRAFO ÚNICO, ART. 2º, DO DECRETO-LEI N.30, DE 15 DE FEVEREIRO DE 1.941."
12792,"DÁ NOVA REDAÇÃO AO ITEM II, DO ARTIGO 2º, DO DECRETO-LEI N. 168, DE 31/12/1.946."


## **Ponto (`.`)**

Os pontos podem ter diversas funcionalidades em um texto:

- **Encerrar frases:** deve ser removido.

- **Separar milhares em números:** deve ser removido, para padronizar os números.

- **Separar as letras em siglas:** deve ser mantido, para evitar interpretação errada da palavra.

- **Indicar abreviações:** se for abreviação importante para nosso contexto, devemos normalizar, senão, deve ser removido.

### **Normalizando o separador de milhar**

In [35]:
# Ponto como separador de milhar
mask = df_clean['summary'].str.contains(r'\d[\.]\d', regex=True)
df_clean.loc[mask]

,summary
5,"(Altera a redação dos artigos 7º, 8º, 9º e 10, da Lei Municipal nº 11.982 , de 14 de maio de 2019 e dá outras providências)."
16,"(Altera a redação do art. 2º, da Lei Municipal nº 12.714 , de 28 de dezembro de 2022, que dispõe sobre normas para execução de serviços de Transporte Escolar Gratuito aos alunos matriculados na Rede Municipal de Ensino de Sorocaba)."
21,"(Altera a Lei nº 12.590 , de 15 de junho de 2022, que reconhece no âmbito do Município de Sorocaba, o Cordão de Girassol como instrumento auxiliar de orientação para identificação de pessoas com deficiência não visível)."
22,"(Dispõe sobre a instituição de Área de Especial Interesse Social para Habitação (AEIS), na área pública remanescente denominada por I-D, da planta de desdobro elaborada pela Prefeitura Municipal de Sorocaba, situada na Avenida Itavuvu localizado no perímetro urbano desta cidade, conforme descrição constante na matrícula nº 125.524 registrada nº 1º Oficial de Registro de Imóveis de Sorocaba para promoção de Lotes Sociais e dá outras providências)."
25,"(Altera dispositivos da Lei nº 12.099 , de 22 de outubro de 2019, que estabelece diretrizes e incentivos fiscais para o desenvolvimento econômico do Município e dá outras providências)."
...,...
12797,"DISPÕE SÔBRE A ABERTURA DE CRÉDITO ESPECIAL DE CR$ 14.583,910."
12798,"DISPÕE SÔBRE ABERTURA DE UM CRÉDITO SUPLEMENTAR DE CR$ 137.000,00."
12804,ORÇA A RECEITA E FIXA A DESPESA DO MUNICÍPIO DE SOROCABA PARA O EXERCÍCIO DE 1.948.
12806,"DÁ NOVA REDAÇÃO AO § 2º, DO ART. 1º, DO DECRETO-LEI Nº 167, DE 31 DE DEZEMBRO DE 1.946"


In [36]:
def normalize_thousand_separator(text):
    pattern = r"(\d)\.(\d)"
    repl = lambda m: m.group().replace('.', '')
    return re.sub(pattern, repl, text)

In [37]:
# Teste de conversão
df_clean.loc[mask, 'summary'].apply(normalize_thousand_separator).to_frame()

,summary
5,"(Altera a redação dos artigos 7º, 8º, 9º e 10, da Lei Municipal nº 11982 , de 14 de maio de 2019 e dá outras providências)."
16,"(Altera a redação do art. 2º, da Lei Municipal nº 12714 , de 28 de dezembro de 2022, que dispõe sobre normas para execução de serviços de Transporte Escolar Gratuito aos alunos matriculados na Rede Municipal de Ensino de Sorocaba)."
21,"(Altera a Lei nº 12590 , de 15 de junho de 2022, que reconhece no âmbito do Município de Sorocaba, o Cordão de Girassol como instrumento auxiliar de orientação para identificação de pessoas com deficiência não visível)."
22,"(Dispõe sobre a instituição de Área de Especial Interesse Social para Habitação (AEIS), na área pública remanescente denominada por I-D, da planta de desdobro elaborada pela Prefeitura Municipal de Sorocaba, situada na Avenida Itavuvu localizado no perímetro urbano desta cidade, conforme descrição constante na matrícula nº 125524 registrada nº 1º Oficial de Registro de Imóveis de Sorocaba para promoção de Lotes Sociais e dá outras providências)."
25,"(Altera dispositivos da Lei nº 12099 , de 22 de outubro de 2019, que estabelece diretrizes e incentivos fiscais para o desenvolvimento econômico do Município e dá outras providências)."
...,...
12797,"DISPÕE SÔBRE A ABERTURA DE CRÉDITO ESPECIAL DE CR$ 14583,910."
12798,"DISPÕE SÔBRE ABERTURA DE UM CRÉDITO SUPLEMENTAR DE CR$ 137000,00."
12804,ORÇA A RECEITA E FIXA A DESPESA DO MUNICÍPIO DE SOROCABA PARA O EXERCÍCIO DE 1948.
12806,"DÁ NOVA REDAÇÃO AO § 2º, DO ART. 1º, DO DECRETO-LEI Nº 167, DE 31 DE DEZEMBRO DE 1946"


### **Ponto como Separador de Siglas**

Neste caso, não vamos fazer nada.

In [38]:
# Ponto como separador de letras maiúsculas (sigla)
df_clean[df_clean['summary'].str.contains(r'[A-Z][\.][A-Z]', regex=True)][['summary']]

,summary
214,(Declara de utilidade pública a Associação Classic Sports - A.C.S e dá outras providências).
1612,AUTORIZA O PODER EXECUTIVO A CONTRATAR OPERAÇÃO DE CRÉDITO JUNTO AO BANCO DO BRASIL S.A. E DÁ OUTRAS PROVIDÊNCIAS CORRELATAS.
1614,"DISPÕE SOBRE A REVOGAÇÃO DA LEI Nº 5.197 , DE 5 DE SETEMBRO DE 1996, QUE DISPÕE SOBRE O ARMAZENAMENTO DE BOTIJÕES DE GÁS LIQUEFEITO DE PETRÓLEO - G.L.P."
1892,"ALTERA A REDAÇÃO DO § 5º, DO ART. 1º, DA LEI Nº 6.294 , DE 13 DE OUTUBRO DE 2000, QUE DISPÕE SOBRE OBRIGATORIEDADE DE INSTALAÇÃO DE S.P.D.A. - SISTEMA DE PROTEÇÃO CONTRA DESCARGAS ATMOSFÉRICAS (PARA-RAIOS) NORMATIZADO E SUBSTITUIÇÃO E RETIRADA DE PARA-RAIOS RADIOATIVOS, ACRESCE DISPOSITIVO À MESMA E DÁ OUTRAS PROVIDÊNCIAS."
2303,"DISPÕE SOBRE DENOMINAÇÃO DE ""FERNANDO DINI NETO"" AO VIADUTO DO COMPLEXO GOVERNADOR FRANCO MONTORO QUE LIGA AS VIAS J.J. LACERDA A AV. IPANEMA."
...,...
11532,"DENOMINA ""PROF.A. MARY DORA D`ARIENZO FAVORETO"", UMA VIA PUBLICA DA CIDADE."
11825,"AUTORIZA O PREFEITO A RESCINDIR OS CONTRATOS DE PAVIMENTAÇÃO ASFÁLTICA COM A FIRMA AGRO-PAVI S.A., AGRICULTURA E ENGENHARIA, AMIGÁVELMENTE, BEM COMO ABRE CRÉDITO ESPECIAL NA SECRETARIA DA FAZENDA MUNICIPAL."
11840,"AUTORIZA O PREFEITO MUNICIPAL A ENTRAR EM ENTENDIMENTO COM A SÃO PAULO SERVIÇOS DE ELETRICIDADE S.A., PARA O FORNECIMENTO DE ENERGIA ELÉTRICA DOMICILIAR, E DÁ OUTRAS PROVIDÊNCIAS."
11895,"AUTORIZA A PREFEITURA MUNICIPAL DE SOROCABA A AVALIZAR UMA NOTA PROMISSÓRIA NO VALOR DE CR$ 60.000.000,00 (SESSENTA MILHÕES DE CRUZEIROS) EM FAVOR DA C.R.T. SOROCABANA, PARA O BANCO DO ESTADO DE SÃO PAULO, NAS CONDIÇÕES QUE MENCIONA."


### **Normalizando as abreviações**

Separar abreviações de finais de frase automaticamente é muito difícil.

Assim, vamos somente padronizar algumas abreviações comuns em textos legislativos que trazem contexto para nossa análise:
- N., Nº ou N.º (número)
- Ns., Nºs ou Nº s (números)
- Art. (artigo)
- Arts. (artigos)

Outras abreviações comuns:
- Prof. (professor)
- Profª, Prof.ª, Profa. ou Prof.a (professora)
- Dr. (doutor)
- Dra. (doutora)
- Av. (avenida)
- Jd. (jardim)
- Pq. (parque)
- Cel. (coronel)
- Cap. (capitão)
- D. ou Da. (dona)
- Sr. (senhor)
- Sra. (senhora)
- Sto (santo)
- Eng. (engenheiro)

Os demais pontos podem ser removidos sem grandes prejuízos.

In [39]:
mask = df_clean['summary'].str.contains(r'[^\w\.][A-Z]\.\s', regex=True, case=False)
df_clean.loc[mask]

,summary
4184,"DISPÕE SOBRE DENOMINAÇÃO DE ""OSÓRIO T. MORAES"" AO TEATRO DE ARENA DO PARQUE DAS ÁGUAS DO ABAETÉ E DÁ OUTRAS PROVIDÊNCIAS."
9711,DECLARA DE UTILIDADE PÚBLICA O E. C. PARANAZINHO.
10486,DECLARA DE UTILIDADE PÚBLICA A CAIXA BENEFICENTE DO HOSPITAL D. LEONOR MENDES DE BARROS.
11252,"AUTORIZA A PREFEITURA MUNICIPAL A CELEBRAR CONVÊNIO COM O GOVÊRNO DO ESTADO DE S. PAULO, PARA RECEBER, EM COMODATO, MATERIAL DE PARQUES INFANTIS."
11283,"PRORROGA PRAZO PREVISTO NO ARTIGO 3º DA LEI N. 1.514 , DE 23 DE OUTUBRO DE 1.968."
11365,"DISPÕE SÔBRE ALTERAÇÕES DA LEI N. 1.444 , DE 13 DE DEZEMBRO DE 1.966, QUE REGULOU O SISTEMA TRIBUTÁRIO MUNICIPAL."
11382,"ALTERA A REDAÇÃO DO ARTIGO 2º, DA LEI N. 845 , DE 10 DE OUTUBRO DE 1.961."
11403,"DISPÕE SÔBRE REVOGAÇÃO DA LEI N. 1.104 , DE 12 DE JUNHO DE 1.963."
11407,"DISPÕE SÔBRE ALTERAÇÃO DAS MULTAS PREVISTAS NOS ARTIGOS 1º E 2º DA LEI N. 1.271 , DE 3 DE NOVEMBRO DE 1.964."
11465,"DISPÕE SÔBRE ELEVAÇÃO DA IMPORTÂNCIA ESTABELECIDA NO ARTIGO 243, DA LEI N. 179, DE 29 DE NOVEMBRO DE 1950."


In [40]:
def expand_abbreviations(text):
    text = re.sub(r'(?P<pre>[^\w\.])(?:Ns\.|Nº\s*s)(?P<número>\s*\d+)', lambda m: m.group('pre') + ' Nºs ' + m.group('número'), text, flags=re.IGNORECASE)
    text = re.sub(r'(?P<pre>[^\w\.])(?:N\.|Nº|N\.º)(?P<número>\s*\d+)', lambda m: m.group('pre') + ' Nº ' + m.group('número'), text, flags=re.IGNORECASE) 
    text = re.sub(r'(?P<pre>[^\w\.])Arts\.', lambda m: m.group('pre') + ' artigos ', text, flags=re.IGNORECASE)
    text = re.sub(r'(?P<pre>[^\w\.])Art\.', lambda m: m.group('pre') + ' artigo ', text, flags=re.IGNORECASE)
    text = re.sub('(?P<pre>[^\w\.])(?:prof\.a|prof\.ª|profª\.?|profa\.)', lambda m: m.group('pre') + ' professora ', text, flags=re.IGNORECASE)
    text = re.sub('(?P<pre>[^\w\.])prof\.', lambda m: m.group('pre') + ' professor ', text, flags=re.IGNORECASE)
    text = re.sub('(?P<pre>[^\w\.])dra\.', lambda m: m.group('pre') + ' doutora ', text, flags=re.IGNORECASE)
    text = re.sub('(?P<pre>[^\w\.])dr\.', lambda m: m.group('pre') + ' doutor ', text, flags=re.IGNORECASE)
    text = re.sub('(?P<pre>[^\w\.])av\.', lambda m: m.group('pre') + ' avenida ', text, flags=re.IGNORECASE)
    text = re.sub('(?P<pre>[^\w\.])jd\.', lambda m: m.group('pre') + ' jardim ', text, flags=re.IGNORECASE)
    text = re.sub('(?P<pre>[^\w\.])pq\.', lambda m: m.group('pre') + ' parque ', text, flags=re.IGNORECASE)
    text = re.sub('(?P<pre>[^\w\.])cel\.', lambda m: m.group('pre') + ' coronel ', text, flags=re.IGNORECASE)
    text = re.sub('(?P<pre>[^\w\.])cap\.', lambda m: m.group('pre') + ' capitão ', text, flags=re.IGNORECASE)
    text = re.sub('(?P<pre>[^\w\.])(?:d\.|da\.)\s', lambda m: m.group('pre') + ' dona ', text, flags=re.IGNORECASE)
    text = re.sub('(?P<pre>[^\w\.])sr\.', lambda m: m.group('pre') + ' senhor ', text, flags=re.IGNORECASE)
    text = re.sub('(?P<pre>[^\w\.])sra\.', lambda m: m.group('pre') + ' senhora ', text, flags=re.IGNORECASE)
    text = re.sub('(?P<pre>[^\w\.])eng\.', lambda m: m.group('pre') + ' engenheiro ', text, flags=re.IGNORECASE)
    return text

In [41]:
# Testando a normalização
df_clean.loc[mask, 'summary'].apply(expand_abbreviations).to_frame()

,summary
4184,"DISPÕE SOBRE DENOMINAÇÃO DE ""OSÓRIO T. MORAES"" AO TEATRO DE ARENA DO PARQUE DAS ÁGUAS DO ABAETÉ E DÁ OUTRAS PROVIDÊNCIAS."
9711,DECLARA DE UTILIDADE PÚBLICA O E. C. PARANAZINHO.
10486,DECLARA DE UTILIDADE PÚBLICA A CAIXA BENEFICENTE DO HOSPITAL dona LEONOR MENDES DE BARROS.
11252,"AUTORIZA A PREFEITURA MUNICIPAL A CELEBRAR CONVÊNIO COM O GOVÊRNO DO ESTADO DE S. PAULO, PARA RECEBER, EM COMODATO, MATERIAL DE PARQUES INFANTIS."
11283,"PRORROGA PRAZO PREVISTO NO ARTIGO 3º DA LEI Nº 1.514 , DE 23 DE OUTUBRO DE 1.968."
11365,"DISPÕE SÔBRE ALTERAÇÕES DA LEI Nº 1.444 , DE 13 DE DEZEMBRO DE 1.966, QUE REGULOU O SISTEMA TRIBUTÁRIO MUNICIPAL."
11382,"ALTERA A REDAÇÃO DO ARTIGO 2º, DA LEI Nº 845 , DE 10 DE OUTUBRO DE 1.961."
11403,"DISPÕE SÔBRE REVOGAÇÃO DA LEI Nº 1.104 , DE 12 DE JUNHO DE 1.963."
11407,"DISPÕE SÔBRE ALTERAÇÃO DAS MULTAS PREVISTAS NOS ARTIGOS 1º E 2º DA LEI Nº 1.271 , DE 3 DE NOVEMBRO DE 1.964."
11465,"DISPÕE SÔBRE ELEVAÇÃO DA IMPORTÂNCIA ESTABELECIDA NO ARTIGO 243, DA LEI Nº 179, DE 29 DE NOVEMBRO DE 1950."


### **Removendo pontos finais**

In [42]:
def remove_end_points(text):
    return re.sub(r'\.\s|\.$', '', text)

In [43]:
# Teste de remoção de pontos finais
mask = df_clean['summary'].str.contains(r'\.\s|\.$', regex=True)
df_clean.loc[mask, 'summary'].apply(remove_end_points).to_frame()

,summary
0,"(Dispõe sobre denominação de ""JOSÉ ANTONIO PASCOTO"" a uma via pública)"
1,"(Dispõe sobre denominação de ""Vitor Hage"" a uma via pública e dá outras providências)"
2,"(Dispõe sobre denominação de ""LISARDO CUNHA DIAS"" a uma via pública e dá outras providências)"
3,"(Dispõe sobre denominação de ""Desirre Ferraz Cardoso"" a uma via pública de nossa cidade e dá outras providências)"
4,"(Institui, no calendário oficial do Município o Dia do Sociólogo e dá outras providências)"
...,...
12798,"DISPÕE SÔBRE ABERTURA DE UM CRÉDITO SUPLEMENTAR DE CR$ 137.000,00"
12799,DISPÕE SÔBRE AUMENTO DE VENCIMENTOS
12800,DISPÕE SÔBRE EXECUÇÃO DE CALÇAMENTO E DÁ OUTRAS PROVIDÊNCIAS
12804,ORÇA A RECEITA E FIXA A DESPESA DO MUNICÍPIO DE SOROCABA PARA O EXERCÍCIO DE 1.948


### **Barra (`/`)**

As barras são usadas para diferentes contextos:

- Separar dias, meses e anos em datas
- Informar o ano de publicação de uma lei
- Separar nomes e siglas
- e/ou

Podemos manter todas elas, caso não estejam isoladas. Caso estejam, podemos remover.

In [44]:
# Barras separando caracteres não numéricos
mask = df_clean['summary'].str.contains(r'\D/\D', regex=True)
df_clean.loc[mask]

,summary
23,"(Institui, no âmbito do Município de Sorocaba, o Programa de Melhorias Habitacionais para reforma de moradias inseridas em Zonas ou Áreas de Especial Interesse Social - AEIS/ZEIS e dá outras providências)."
84,"(Autoriza a concessão de Subvenções às entidades sem fins lucrativos que complementam o SUS, decorrentes da transposição e transferência dos saldos financeiros remanescentes de exercícios anteriores a 2018, conforme Portaria GM/MS nº 96, de 7 de fevereiro de 2023)."
90,"(Institui o Plano Municipal de Combate às Drogas no Município de Sorocaba e proíbe a comercialização, exibição ou divulgação, em quaisquer estabelecimentos de livros, revistas, jornais, matérias publicitárias ou materiais congêneres que façam apologia à posse para consumo e uso pessoal de substâncias entorpecentes e/ou psicotrópicas ilícitas, que possam causar dependência, e dá outras providências)."
92,"Estabelece desconto sobre o valor da tarifa mínima mensal de serviço de água, em caso de fornecimento sem observâncias às normas relativas à qualidade no Município de Sorocaba/SP."
94,"(Dispõe sobre a proibição da realização de marchas, inclusive Marcha da Maconha, eventos, reuniões, ou práticas análogas, que façam apologia à posse para consumo e uso pessoal, relativas a substâncias ilícitas e ilegítimas entorpecentes e/ou psicotrópicas, que possam causar dependência, bem como dá outras providências)."
...,...
11221,DISPÕE SÔBRE DOAÇÃO DE ÁREA DE TERRENO À DRURY´S S/A DISTRIBUIDORA DE PRODUTOS INTERNACIONAIS.
11268,"DISPÕE SÔBRE ABERTURA DE CRÉDITO ESPECIAL, DESTINADO À AQUISIÇÃO DE IMÓVEL A SER DOADO À ""FÁBRICA DE AÇO PAULISTA S/A"", NOS TÊRMOS DA LEI MUNICIPAL 1.543, DE 15/1/69."
11889,"DISPÕE SÔBRE CRIAÇÃO DE COMISSÃO MISTA, DO EXECUTIVO E LEGISLATIVO, PARA ENTRAR EM ENTENDIMENTOS COM A SÃO PAULO - SERVIÇOS DE ELETRICIDADE S/A, PARA RESCISÃO DE CONTRATO AMIGAVELMENTE."
11933,"DISPÕE SÔBRE AUTORIZAÇÃO À PREFEITURA MUNICIPAL PARA EMITIR NOTAS PROMISSÓRIAS, PARA PAGAMENTO DE DÉBITO COM A SÃO PAULO SERVIÇOS DE ELETRICIDADE S/A."


In [45]:
# Barras separando números, com espaços
mask = df_clean['summary'].str.contains(r'\d\s/\d', regex=True)
df_clean.loc[mask]

# Neste caso, precisamos remover os espaços
# Exemplo: 11.340 /2006 -> 11.340/2006

,summary
265,"(Acrescenta a alínea ""k"" ao inciso II do art. 1º da Lei nº 10.128 , de 30 de maio de 2012, para incluir a vedação de nomeação pelo Poder Executivo e Legislativo do Município de Sorocaba de pessoas condenadas pela Lei Federal nº 11.340 /2006 - Lei Maria da Penha)."
439,Dispõe sobre o credenciamento de administradoras e operadoras de planos de saúde aos servidores contratados sob o regime da Consolidação das Leis do Trabalho - CLT e Conselheiros Tutelares e seus dependentes e dá outras providência. (Redação dada pela Lei nº 12.536 /2022)
507,"(Altera a Lei nº 4.619 /1994, para acrescentar a previsão de Tombamento de Fachada)."
656,Cria a Patrulha Ambiental/Animal e institui a Gratificação Prêmio de Fiscalização do Meio Ambiente (GPFMA) e dá outras providências. (Redação dada pela Lei nº 12.381 /2021)
745,"Dispõe sobre a criação no site da Prefeitura Municipal de Sorocaba de um link no menu para acompanhamento das emendas dos vereadores à lei orçamentária, desde a sua aprovação até a sua conclusão, visando maior transparência e publicidade, conforme a Lei Federal Complementar 131 /2009 da Transparência e a Lei Federal 12.527 /2011 de Acesso à Informação."
...,...
11653,"DISPÕE SÔBRE CRIAÇÃO DE UM PARQUE INFANTIL EM MARIA HELENA DE MORAIS , E DÁ OUTRAS PROVIDÊNCIAS. (Alteração dada pela Lei 1290 /1964)"
11758,"DISPÕE SÔBRE DENOMINAÇÃO DE ""LYSCIO BRANDÃO DE CAMARGO"" A UMA VIA PÚBLICA DA CIDADE. (Alteração dada pela Lei nº 5753 /1998)"
12015,"DISPÕE SÔBRE DENOMINAÇÃO DE ""RAMON HARO MARTINI"", A UMA VIA PÚBLICA DA CIDADE. (Alteração dada pela Lei nº 5616 /1998)"
12198,"DISPÕE SÔBRE DENOMINAÇÃO DE ""ANGELINA PEROLINA ZOCCA"" , A UMA VIA PÚBLICA DA CIDADE. (Alteração dada pela Lei nº 5641 /1998)"


In [46]:
# Barras isoladas no texto
mask = df_clean['summary'].str.contains(r'\s/\s', regex=True)
df_clean.loc[mask]

# Neste caso, as barras não adicionam contexto, então serão removidas

,summary
279,"Institui programa de apoio aos Micro empreendedores individuais (MEI), Micro e Pequenas Empresas, com isenção do pagamento de taxa de fiscalização de instalação e funcionamento / taxa de publicidade durante a pandemia causada pelo coronavírus (COVID-19) no município de Sorocaba e dá outras providências."
9828,"INSTITUI O PRÊMIO / ""PROFESSOR FLÁVIO GAGLIARDI"", DE ARTES PLÁSTICAS E DÁ OUTROS PROVIDÊNCIAS."


### **Normalizando as barras**

In [47]:
def normalize_slashes(text):
    pattern = r'(\d)\s(/\d)|(\d/)\s(\d)'
    repl = lambda m: m.group().replace(' ', '')
    return re.sub(pattern, repl, text).replace(' / ', ' ')

In [48]:
# Teste de normalização de barras
mask = df_clean['summary'].str.contains(r'\s/\s|\d\s/\d', regex=True)
df_clean.loc[mask, 'summary'].apply(normalize_slashes).to_frame()

,summary
265,"(Acrescenta a alínea ""k"" ao inciso II do art. 1º da Lei nº 10.128 , de 30 de maio de 2012, para incluir a vedação de nomeação pelo Poder Executivo e Legislativo do Município de Sorocaba de pessoas condenadas pela Lei Federal nº 11.340/2006 - Lei Maria da Penha)."
279,"Institui programa de apoio aos Micro empreendedores individuais (MEI), Micro e Pequenas Empresas, com isenção do pagamento de taxa de fiscalização de instalação e funcionamento taxa de publicidade durante a pandemia causada pelo coronavírus (COVID-19) no município de Sorocaba e dá outras providências."
439,Dispõe sobre o credenciamento de administradoras e operadoras de planos de saúde aos servidores contratados sob o regime da Consolidação das Leis do Trabalho - CLT e Conselheiros Tutelares e seus dependentes e dá outras providência. (Redação dada pela Lei nº 12.536/2022)
507,"(Altera a Lei nº 4.619/1994, para acrescentar a previsão de Tombamento de Fachada)."
656,Cria a Patrulha Ambiental/Animal e institui a Gratificação Prêmio de Fiscalização do Meio Ambiente (GPFMA) e dá outras providências. (Redação dada pela Lei nº 12.381/2021)
...,...
11653,"DISPÕE SÔBRE CRIAÇÃO DE UM PARQUE INFANTIL EM MARIA HELENA DE MORAIS , E DÁ OUTRAS PROVIDÊNCIAS. (Alteração dada pela Lei 1290/1964)"
11758,"DISPÕE SÔBRE DENOMINAÇÃO DE ""LYSCIO BRANDÃO DE CAMARGO"" A UMA VIA PÚBLICA DA CIDADE. (Alteração dada pela Lei nº 5753/1998)"
12015,"DISPÕE SÔBRE DENOMINAÇÃO DE ""RAMON HARO MARTINI"", A UMA VIA PÚBLICA DA CIDADE. (Alteração dada pela Lei nº 5616/1998)"
12198,"DISPÕE SÔBRE DENOMINAÇÃO DE ""ANGELINA PEROLINA ZOCCA"" , A UMA VIA PÚBLICA DA CIDADE. (Alteração dada pela Lei nº 5641/1998)"


### **Símbolo de Seção ou Parágrafo (`§`)**

Símbolos de parágrafos normalmente estão associados a alterações de leis já existentes. Vamos mantê-los.

In [49]:
mask = df_clean['summary'].str.contains(r'§', regex=True)
df_clean.loc[mask]

,summary
61,"(Altera o § 2º, do artigo 2º, da Lei nº 10.710 , de 8 de janeiro de 2014, que autoriza o fechamento das vilas e ruas sem saída residenciais ao tráfego de veículos estranhos aos seus moradores)."
156,"(Altera a redação do § 1º, do artigo 2º e do caput, do artigo 3º, da Lei nº 8.336 , de 19 de dezembro de 2007, que dispõe sobre a criação de Fundos de Previdência Municipal e dá outras providências)."
233,"(Altera a redação do § 3º do art. 2º da Lei nº 12.308 , de 28 de maio de 2021, e dá outras providências)."
247,"(Altera a redação do inciso IV e acrescenta § 4º ao art. 125 da Lei nº 10.060 , de 3 de maio de 2012 que dispõe sobre a Política Municipal de Meio Ambiente de Sorocaba e dá outras providências)."
333,"(Adiciona o § 3º ao artigo 1º da Lei nº 11.449 , de 7 de novembro de 2016)."
...,...
12341,"DÁ NOVA REDAÇÃO AO ARTIGO 66 E SEU § 1º DA LEI Nº 162 , DE 18 DE AGOSTO DE 1950."
12379,"DÁ NOVA REDAÇÃO AO § 1º, DO CÓDIGO DE OBRAS (LEI Nº 162 ), DO MUNICÍPIO."
12452,"ALTERA O § ÚNICO DO ARTIGO 1º, DA LEI Nº 175."
12699,"REVOGA O § ÚNICO DO ART. 2º, DA LEI Nº 99 , E DÁ OUTRAS PROVIDÊNCIAS."


### **Símbolo ordinal masculino (`º`)**

Está associado a números de leis e artigos. Em boa parte, aparece em alterações de leis existentes. Vamos manter somente quando está na abreviação `Nº` e remover dos demais. 

In [50]:
mask = df_clean['summary'].str.contains(r'º', regex=True)
df_clean.loc[mask]

,summary
5,"(Altera a redação dos artigos 7º, 8º, 9º e 10, da Lei Municipal nº 11.982 , de 14 de maio de 2019 e dá outras providências)."
16,"(Altera a redação do art. 2º, da Lei Municipal nº 12.714 , de 28 de dezembro de 2022, que dispõe sobre normas para execução de serviços de Transporte Escolar Gratuito aos alunos matriculados na Rede Municipal de Ensino de Sorocaba)."
21,"(Altera a Lei nº 12.590 , de 15 de junho de 2022, que reconhece no âmbito do Município de Sorocaba, o Cordão de Girassol como instrumento auxiliar de orientação para identificação de pessoas com deficiência não visível)."
22,"(Dispõe sobre a instituição de Área de Especial Interesse Social para Habitação (AEIS), na área pública remanescente denominada por I-D, da planta de desdobro elaborada pela Prefeitura Municipal de Sorocaba, situada na Avenida Itavuvu localizado no perímetro urbano desta cidade, conforme descrição constante na matrícula nº 125.524 registrada nº 1º Oficial de Registro de Imóveis de Sorocaba para promoção de Lotes Sociais e dá outras providências)."
25,"(Altera dispositivos da Lei nº 12.099 , de 22 de outubro de 2019, que estabelece diretrizes e incentivos fiscais para o desenvolvimento econômico do Município e dá outras providências)."
...,...
12699,"REVOGA O § ÚNICO DO ART. 2º, DA LEI Nº 99 , E DÁ OUTRAS PROVIDÊNCIAS."
12703,"DISPÕE SÔBRE ABERTURA DE UM CRÉDITO ESPECIAL, PARA ATENDER AS DESPESAS COM A EXECUÇÃO DOS SERVIÇOS AUTORIZADOS PELA LEI Nº 62."
12746,"ALTERA O DISPOSTO NA LETRA ""A"" DO PARÁGRAFO ÚNICO, ART. 2º, DO DECRETO-LEI N.30, DE 15 DE FEVEREIRO DE 1.941."
12792,"DÁ NOVA REDAÇÃO AO ITEM II, DO ARTIGO 2º, DO DECRETO-LEI N. 168, DE 31/12/1.946."


In [51]:
def normalize_ordinal_indicator(text):
    pattern = r'(\d)º'
    repl = lambda m: m.group(1)
    return re.sub(pattern, repl, text)

# **Outras transformações**


- Nos casos de `S/A` e `S.A`, vamos normalizar para `S.A`

- Nos casos de `e/ou`, vamos normalizar para `e ou`

- Nos casos de datas `\d{1,2}/\d{1,2}/\d{2,4}` e `DD de MMM de YYYY`, vamos normalizar para `TOKEN_DATA`

- Nos casos de `Nº \d+/\d+`, vamos normalizar para `Nº \d+ de TOKEN_DATA`

- Nos casos de `\n`, vamos substituir por `\s`

- Conversão para letras minúsculas

- Remover espaços excedentes

- Normalizar textos de acordo com novo acordo ortográfico

- Substituir qualquer número por `#`

In [52]:
def convert_to_lowercase(text):
    return text.lower()

In [53]:
def strip_whitespace(text):
    return re.sub(r'\s+', ' ', text).strip()

In [54]:
def normalize_dates(text):
    pattern1 = r'\d{1,2}/\d{1,2}/\d{2,4}'
    pattern2 = r'\d{1,2}\s+de\s(?:janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)\s+de\s+(?:\d{2,4}|\d,\d{3})'
    pattern3 = r'(Nº\s\d+)/(?:\d{2,4}|\d,\d{3})'
    
    repl = lambda m: ' TOKEN_DATA '
    repl3 = lambda m: m.group(1) + ' de TOKEN_DATA'
    
    text = re.sub(pattern1, repl, text)
    text = re.sub(pattern2, repl, text, flags=re.IGNORECASE)
    text = re.sub(pattern3, repl3, text, flags=re.IGNORECASE)
    return text

In [55]:
def remove_new_lines(text):
    return text.replace('\n', ' ')

In [56]:
def normalize_societies(text):
    return re.sub(r'\sS/A\s', ' S.A ', text, flags=re.IGNORECASE)

In [57]:
def normalize_and_or(text):
    return re.sub(r'\se/ou\s', ' e ou ', text, flags=re.IGNORECASE)

In [58]:
def fix_ortography(text):
    text = text.replace('sôbre', 'sobre')
    text = text.replace('providencias', 'providências')
    text = text.replace('decreto lei', 'decreto-lei')
    text = text.replace('govêrno', 'governo')
    text = text.replace('qü', 'qu')
    text = text.replace('gü', 'gu')
    text = text.replace('éia ', 'eia ')
    text = text.replace('ôo ', 'oo ')
    return text

In [59]:
def normalize_numbers(text):
    return re.sub(r'\d+', ' # ', text)

# **Aplicando tudo**

Vamos agora aplicar todas as transformações, na ordem correta, de forma a podermos aproveitar transformações anteriores para facilitar as subsequentes.

In [60]:
def transform_text(text):
    text = remove_unnecessary_chars(text)
    text = remove_new_lines(text)
    text = normalize_ampersand(text)
    text = normalize_dashes(text)
    text = normalize_ordinal_indicator(text)
    text = convert_to_lowercase(text)
    text = normalize_money(text)
    text = normalize_thousand_separator(text)
    text = normalize_commas(text)
    text = normalize_slashes(text)
    text = normalize_and_or(text)
    text = normalize_dates(text)
    text = normalize_societies(text)
    text = expand_abbreviations(text)
    text = remove_end_points(text)
    text = strip_whitespace(text)
    text = fix_ortography(text)
    text = normalize_numbers(text)
    return text
    

In [61]:
df_clean['summary'] = df_clean['summary'].apply(transform_text)

In [62]:
df_clean

,summary
0,dispõe sobre denominação de josé antonio pascoto a uma via pública
1,dispõe sobre denominação de vitor hage a uma via pública e dá outras providências
2,dispõe sobre denominação de lisardo cunha dias a uma via pública e dá outras providências
3,dispõe sobre denominação de desirre ferraz cardoso a uma via pública de nossa cidade e dá outras providências
4,institui no calendário oficial do município o dia do sociólogo e dá outras providências
...,...
12807,dispõe sobre abertura de crédito especial de $ TOKEN_VALOR
12808,dispõe sobre concessão de auxílio e dá outras providências
12809,dispõe sobre concessão de licença prêmio aos funcionários municipais
12810,dispõe sobre apreensão e eliminação de animais


In [63]:
# Nova contagem de caracteres especiais
special_chars = string.punctuation + '§ºª°´'
for c in special_chars:
    print(c, sum(df_clean['summary'].str.contains(c, regex=False)))

! 0
" 0
# 2530
$ 61
% 13
& 1
' 0
( 0
) 0
* 0
+ 0
, 4
- 365
. 124
/ 86
: 0
; 0
< 0
= 0
> 0
? 0
@ 0
[ 0
\ 0
] 0
^ 0
_ 2006
` 0
{ 0
| 0
} 0
~ 0
§ 132
º 2066
ª 0
° 0
´ 0


In [64]:
df_clean.loc[df_clean['summary'].str.contains('#', regex=True)]

,summary
5,altera a redação dos artigos # # # e # da lei municipal Nº # de TOKEN_DATA e dá outras providências
7,institui e insere no calendário oficial do município de sorocaba a semana municipal do dia # de abril como a semana municipal de conscientização da doença de parkinson denominada tulipa vermelha
13,fica instituído no calendário oficial do município de sorocaba o dia pela valorização à prática da vacinação a ser celebrada anualmente em # de outubro
16,altera a redação do artigo # da lei municipal Nº # de TOKEN_DATA que dispõe sobre normas para execução de serviços de transporte escolar gratuito aos alunos matriculados na rede municipal de ensino de sorocaba
20,fica instituído no calendário oficial do município de sorocaba o dia do combate e prevenção ao câncer infantojuvenil a ser celebrado anualmente em # de fevereiro
...,...
12746,altera o disposto na letra a do parágrafo único artigo # do decreto-lei Nº # de TOKEN_DATA
12747,modifica o decreto-lei Nº # de TOKEN_DATA e a lei Nº # de TOKEN_DATA sobre a taxa de licença pública
12792,dá nova redação ao item ii do artigo # do decreto-lei Nº # de TOKEN_DATA
12804,orça a receita e fixa a despesa do município de sorocaba para o exercício de #


# **Salvando o resultado final**

In [65]:
# Salvar o dataset
df_clean.to_csv('data/Leis de Sorocaba (Clean).csv', index=False)